In [1]:
import pandas as pd
import time

from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [2]:
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service

In [3]:
service = Service(executable_path=GeckoDriverManager().install())

# Set Firefox options (optional)
options = webdriver.FirefoxOptions()

# Initialize the Firefox driver with the service and options
driver = webdriver.Firefox(service=service, options=options)

# Navigate to the Billboard Hot 100 chart
driver.get("https://www.billboard.com/charts/hot-100/")

# Example: Creating a list of years from 2019 to 2023
start_year = 2019
end_year = 2023

# Example: Using a while loop to generate a list of years from start_year to end_year
years = []
year = start_year
while year <= end_year:
    years.append(str(year))
    year += 1

In [4]:
from selenium.webdriver.common.by import By

# Dictionary to store name of artists
artists_data = {}

# Dictionary to store top 100 songs
songs_data = {}

# Initialize index for iterating through dates
index = 0
while index < len(years):
    j = years[index]
    link = "https://www.billboard.com/charts/hot-100/{}-07-01/".format(j)
    driver.implicitly_wait(10)
    driver.get(link)

    # Container for referencing the division with top 100 songs
    div = driver.find_elements(By.CLASS_NAME, 'o-chart-results-list-row-container')
    songs, artists = [], []
    print(j)
    
    # Initialize index for iterating through div elements
    i = 0
    while i < len(div):
        # Getting the song name
        song = div[i].find_element(By.CSS_SELECTOR, 'li ul li h3').text
        # Getting the artist name
        name = div[i].find_element(By.CSS_SELECTOR, 'li ul li span').text
        # Appending the song and artist name to the lists
        songs.append(song)
        artists.append(name)
        
        # Move to the next div element
        i += 1

    artists_data[j] = artists
    songs_data[j] = songs
    
    # Move to the next date
    index += 1

# Terminating a session and closing the active browser
driver.close()

2019
2020
2021
2022
2023


In [5]:
service = Service(executable_path=GeckoDriverManager().install())
options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(service=service, options=options)

In [6]:
# Browsing to Google
driver.get("https://www.google.com")

In [7]:
# List to store the rows of data
rows_data = []

# Fetching song lyrics for top 100 songs
index_years = 0
while index_years < len(songs_data.keys()):
    year = list(songs_data.keys())[index_years]
    songs_list = songs_data[year]
    artists_list = artists_data[year]
    
    print(year)
    index_songs = 0
    while index_songs < len(songs_list):
        song = songs_list[index_songs]
        artist = artists_list[index_songs]
        # Waiting for the google search button element to be clickable
        element = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.NAME, "q")))
        # Entering the search text
        element.send_keys(f"{artist} {song} lyrics")
        # Submitting the form for Google search
        element.submit()
        # Preventing captcha due to bot activity
        time.sleep(10)
        try:
            # Fetching the default lyrics fetched by Google using 'lyricfind'
            lyric = driver.find_element(By.CLASS_NAME, 'xaAUmb').text
            # If default lyrics are available, set lyric status to True
            status = True
        except:
            # If default lyrics are not available, set lyric status to False
            status = False
            lyric = None
        try:
            # Fetching genre and release year of the song
            info = driver.find_elements(By.CSS_SELECTOR, 'div.rVusze')
            release_year = next((i.find_element(By.CLASS_NAME, 'LrzXr').text for i in info if i.find_element(By.CLASS_NAME, 'w8qArf').text == 'Released:'), None)
            genre = next((i.find_element(By.CLASS_NAME, 'LrzXr').text for i in info if i.find_element(By.CLASS_NAME, 'w8qArf').text == 'Genres:'), None)
        except:
            # If genre and year information is not available
            release_year = None
            genre = None
        
        # Appending the row data to the list
        rows_data.append({'Top100Year': year, 'SongTitle': song, 'Artist': artist, 'LyricsStatus': status, 'Lyrics': lyric, 'ReleaseYear': release_year, 'Genre': genre})
        
        # Browsing back to the Google search home page
        driver.back()
        
        # Move to the next song
        index_songs += 1

    # Move to the next year
    index_years += 1

# Terminating the session and closing the active browser

# Creating DataFrame from the list of rows data
df = pd.DataFrame(rows_data)

2019


NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:191:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:662:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.goBack@chrome://remote/content/marionette/driver.sys.mjs:936:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [ ]:
df

In [ ]:
# Converting DataFrame to CSV
df.to_csv('billboard_2019_to_2023_top_100_song_lyrics.csv', index=False)